# Project: NIP
# Subroject: Sylhet Floods, June 2022
## Script: calc_fracArea_NASAIMPACT.ipynb
This script generates a time series of the fractional inundated area, from the outputs of the NASA IMPACT ML Sentinel-1 surface water mapping algorithm.

In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
from itertools import chain
from pyproj import Transformer
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from datetime import datetime
import geopandas as gpd
import copy
import collections
from shapely.geometry import mapping
import pycrs
from shapely.geometry import Polygon
from functools import reduce
import shutil

In [2]:
# Set the root path
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data')

In [3]:
# Add path for the Helpers modules
module_path = os.path.abspath(os.path.join('C:/Users/alexsaunders/Documents/01_uoa/04_git/NIP/Sylhet/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# Import module from analyse_modifiedDevries containing helpful functions to use
import importlib
import Helpers.analyse_modifiedDevries as analyse_DV
importlib.reload(analyse_DV)

<module 'Helpers.analyse_modifiedDevries' from 'C:\\Users\\alexsaunders\\Documents\\01_uoa\\04_git\\NIP\\Sylhet\\Helpers\\analyse_modifiedDevries.py'>

## PART 1: Define the ROI(s) for computing fractional flooded area
Load in the admin areas and gauge locations, create some buffer sizes around the gauges to use as ROI for computing FFA

In [5]:
# Move the mosaic NASSA IMPACT predictions to a single folder
dataPath = rootPath/'Raster/SylhetNASAImpact/5_Preds'

In [6]:
dataDatePaths = list(dataPath.iterdir())[:-1]

In [14]:
mosaicPath=dataPath/'Mosaic/v2'
mosaicPath.mkdir(exist_ok=True)

In [13]:
# for dataDatePath in dataDatePaths:
#     mosaicFile=list((dataDatePath/'Mosaic').iterdir())[0]
#     #Make a copy in the central Mosaic folder and delete original file
#     filename=mosaicFile.stem+mosaicFile.suffix
#     shutil.copy(mosaicFile, mosaicPath/filename)
#     # os.remove(mosaicFile)

In [16]:
mosaics = [file for file in mosaicPath.iterdir() if file.is_file() and '2022' in str(file) and file.suffix == '.tif']
mosaics

[WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220501.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220511.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220513.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220523.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220525.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220604.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220606.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220616.tif'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/Mosaic/v2/20220618.tif'),
 WindowsPath('Z:/me

In [8]:
# Import the ROI geometry from shapefile
adminShapePath = rootPath/'Shapefiles/AdminHDX'
districts = gpd.read_file(adminShapePath/'bgd_admbnda_adm2_bbs_20201113.shp')
upazillas = gpd.read_file(adminShapePath/'bgd_admbnda_adm3_bbs_20201113.shp')
sylhet_dist = districts[districts['ADM2_EN']=='Sylhet']
sylhet_upa = upazillas[upazillas['ADM3_EN']=='Sylhet Sadar']

In [9]:
# Or instead create the ROI as a buffer around the gauge locations

# Import the gauge station locations
gaugePath = rootPath/'Stream_Gauge/StreamGaugeShapefiles'
gaugeLocs = gpd.read_file(gaugePath/'streamgaugew_division.shp') # read the file as a geopandas

# Create buffer around all gauges - use several different buffer sizes starting from 100 m, to 1 km
gaugeLocsGeoms = gaugeLocs.geometry
buffer_sizes = [5, 10, 15]#[0.1, 0.2, 0.5, 1]
pd.options.mode.chained_assignment = None  # default='warn'

for buffer_size in buffer_sizes:
    
    # Add new field to locs geodataframe to store the buffer geometries
    gaugeLocs['buffer_'+str(buffer_size)] = 0

    # For each gaugeLoc, get the buffer and add back to the geodataframe
    for i in range(0, len(gaugeLocs)):
        gaugeLocGeom = gaugeLocsGeoms[i]
        buffer = analyse_DV.geodesic_point_buffer(gaugeLocGeom.y, gaugeLocGeom.x, buffer_size) # lat, lon, buffer (in km)
        gaugeLocs['buffer_'+str(buffer_size)][i] = Polygon(buffer)

In [10]:
# Define the gauges in Sylhet District
streamGaugeDataPath = rootPath/'Stream_Gauge/'
gauges_in_sylhet = pd.read_csv(streamGaugeDataPath/'streamgauges_SylhetDist.csv', header=0)
gauges_in_sylhet = list(gauges_in_sylhet['streamvect'])
gaugeDataSummaryMaster = pd.read_csv(streamGaugeDataPath/'CombinedAll2004-2022/Combined/summary.csv', header=0)
gauges = [match for match in gauges_in_sylhet if match in list(gaugeDataSummaryMaster['stationID'][np.logical_and(~np.isnan(gaugeDataSummaryMaster['danger_level']),gaugeDataSummaryMaster['danger_level']<1000)])]
print(gauges)
gauges_in_sylhet_geom = gaugeLocs[gaugeLocs['streamvect'].isin(gauges_in_sylhet)]
gauges_geom = gaugeLocs[gaugeLocs['streamvect'].isin(gauges)]

['SW172.5', 'SW173', 'SW175.5', 'SW251', 'SW266', 'SW267']


In [11]:
gauges_list = list(chain.from_iterable([list(gauges_geom['streamvect']+'_'+str(buffer_size)+'km') for buffer_size in buffer_sizes]))
gauges_geom_list = list(chain.from_iterable([list(gauges_geom['buffer_'+str(buffer_size)]) for buffer_size in buffer_sizes]))

In [12]:
# Create a single geodataframe of ROIs over which to calculate FFA
rois = {'id': ['Sylhet_District', 'Sylhet_Upazilla']+gauges_list, 
        'geometry': list(sylhet_dist.geometry)+list(sylhet_upa.geometry)+gauges_geom_list}
rois_gdf = gpd.GeoDataFrame(rois, geometry='geometry', crs='epsg:4326')

## PART 2: Compute the fractional inundated area at each timestep
For each date, extract the fractional inundated area for the specified ROI.

In [17]:
# Get the dates of images
dates = [str(datetime.strptime(mosaic.stem, '%Y%m%d').date()) for mosaic in mosaics]
print(dates)

['2022-05-01', '2022-05-11', '2022-05-13', '2022-05-23', '2022-05-25', '2022-06-04', '2022-06-06', '2022-06-16', '2022-06-18', '2022-06-28', '2022-06-30', '2022-07-12', '2022-07-22', '2022-07-24', '2022-08-03', '2022-08-05', '2022-08-15', '2022-08-27', '2022-08-29']


In [18]:
# Run the function to create temp raster file for using in masking - ONLY NEEDS TO BE RUN ONCE

#Run the function which creates a raster of values 10, the same dimensions as the flood raster, to be used for clipping to the ROI later
# date1 = '20220604'
# mosaic = [mosaic for mosaic in mosaics if date1 in str(mosaic)][0]
analyse_DV.create_temp_raster_for_mask(mosaics[0], rootPath, mosaicPath) #'Raster/SylhetNASAImpact/5_Preds/Mosaic') # creates raster_tmp

In [19]:
# Load the temp raster in and intersect it with the ROI - this will be used for clipping to the ROI since it contains values of 10
# raster_tmp = rasterio.open(rootPath/'Raster/Sylhet/Sen1MitchellSingleOrbit/Mosaic'/'temp.tif') # can recreate this file if needs for a different ROI area by rerunning the above function
raster_tmp = rasterio.open(mosaicPath/'temp.tif') # can recreate this file if needs for a different ROI area by rerunning the above function

In [21]:
# Run the function to get the fractional flooded area for all image dates, for all the provided ROIs
mosaic_dates = []
pctWaters = []
rois_list = []

# Loop through image dates
for n, mosaic in enumerate(mosaics):
    
    # Get the date
    mosaic_date = str(datetime.strptime(mosaic.stem, '%Y%m%d').date())
    print('Mosaic:', mosaic_date, mosaic)

#     # Load the raster using rioxarray
#     raster = prep_raster.load_raster_for_comparison(mosaic, -10)

#     # Reproject and match the GFM raster to the modified Devries raster to make the processing the same
#     raster_reproj = prep_raster.reproj_match_raster(raster, raster_Devries)
    
    # Load the raster
    with rasterio.open(mosaic) as file:
        raster = file.read(1)

    # Set the water values to 1
    water = copy.copy(raster)
    water[water==1]=1
    
    # Loop through all ROI geometries
    for j in range(len(rois_gdf)):

        # Run the function to create the raster mask for the ROI, using the temp raster containing all values 10
        roiID = rois_gdf.id[j]
        roi = rois_gdf.loc[[j]]
        roi_mask = analyse_DV.create_roi_mask(raster_tmp, roi)

        # Apply the mask to the water_sum raster by performing an array sum operation
        water_roi = np.add(water, roi_mask)

        # Get the number of pixels of water for each date for the ROI only (pixel values in ROI have been shifted by +10 from original values)
        water_roi_vals = list(chain.from_iterable(water_roi.tolist()))
        counter_roi = collections.Counter(water_roi_vals) # 10 = none, 11 = water, other = invalid or outside ROI
        pctWater_roi = counter_roi[11] / (counter_roi[10] + counter_roi[11])
        #print(counter_roi)
        print('ROI % water at {0} for {1}: {2:2.1f}%'.format(mosaic_date, roiID, pctWater_roi*100))
        
        # Record the results for the date and ROI
        mosaic_dates.append(mosaic_date)
        rois_list.append(roiID)
        pctWaters.append(pctWater_roi)

Mosaic: 2022-05-01 Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetNASAImpact\5_Preds\Mosaic\v2\20220501.tif
ROI % water at 2022-05-01 for Sylhet_District: 4.5%
ROI % water at 2022-05-01 for Sylhet_Upazilla: 2.4%
ROI % water at 2022-05-01 for SW172.5_5km: 3.5%
ROI % water at 2022-05-01 for SW173_5km: 4.6%
ROI % water at 2022-05-01 for SW175.5_5km: 5.2%
ROI % water at 2022-05-01 for SW251_5km: 2.4%
ROI % water at 2022-05-01 for SW266_5km: 3.8%
ROI % water at 2022-05-01 for SW267_5km: 2.3%
ROI % water at 2022-05-01 for SW172.5_10km: 3.9%
ROI % water at 2022-05-01 for SW173_10km: 3.9%
ROI % water at 2022-05-01 for SW175.5_10km: 4.3%
ROI % water at 2022-05-01 for SW251_10km: 3.5%
ROI % water at 2022-05-01 for SW266_10km: 3.6%
ROI % water at 2022-05-01 for SW267_10km: 1.8%
ROI % water at 2022-05-01 for SW172.5_15km: 3.7%
ROI % water at 2022-05-01 for SW173_15km: 4.4%
ROI % water at 2022-05-01 for SW175.5_15km: 3.4%
ROI % water at 2022-05-01 for SW251_15km: 4.3%
ROI % water at 2022-05-01 f

In [22]:
# Create dataframe of the results - fractional flooded area
fracFloodArea = pd.DataFrame(data = [mosaic_dates, rois_list, pctWaters], index = ['Date','ROI','FFA']).T
fracFloodArea = fracFloodArea.pivot(index='Date', columns='ROI', values='FFA')
fracFloodArea.index = pd.to_datetime(fracFloodArea.index)

In [24]:
# Export the results as csv
outputPath = rootPath/'Table/SylhetFracFloodedArea'
outputPath.mkdir(exist_ok=True)
pd.DataFrame.to_csv(fracFloodArea, outputPath/'FFA_sylhet_allROI_allGauges_NASAIMPACT_v2.csv', index=True)

In [23]:
fracFloodArea

ROI,SW172.5_10km,SW172.5_15km,SW172.5_5km,SW173_10km,SW173_15km,SW173_5km,SW175.5_10km,SW175.5_15km,SW175.5_5km,SW251_10km,SW251_15km,SW251_5km,SW266_10km,SW266_15km,SW266_5km,SW267_10km,SW267_15km,SW267_5km,Sylhet_District,Sylhet_Upazilla
Date,,,,,,,,,,,,,,,,,,,,
2022-05-01,0.038951,0.037124,0.034695,0.038624,0.043566,0.045891,0.043386,0.034026,0.051989,0.035424,0.043219,0.024194,0.035547,0.033514,0.038202,0.018269,0.01983,0.023176,0.045054,0.024372
2022-05-11,0.242176,0.216392,0.20854,0.121924,0.126908,0.102014,0.068647,0.056731,0.0815,0.114727,0.12886,0.092338,0.073569,0.106735,0.034259,0.156286,0.169964,0.098488,0.149999,0.108618
2022-05-13,0.371452,0.359985,0.266779,0.354804,0.38697,0.344837,0.202325,0.216355,0.158114,0.522125,0.493761,0.495381,0.550531,0.444512,0.551676,0.267537,0.343872,0.125133,0.454619,0.310883
2022-05-23,0.403075,0.455049,0.285776,0.562714,0.550717,0.613673,0.56353,0.545173,0.527561,0.50442,0.523079,0.358228,0.6205,0.498506,0.639306,0.350806,0.45238,0.156991,0.586484,0.445056
2022-05-25,0.293789,0.372447,0.164784,0.490375,0.490636,0.549239,0.536507,0.519359,0.485052,0.432928,0.459394,0.240228,0.524518,0.432523,0.48641,0.293242,0.396385,0.114436,0.530735,0.395344
2022-06-04,0.331226,0.38935,0.21543,0.432013,0.441997,0.48678,0.535609,0.508296,0.523561,0.484093,0.488072,0.347453,0.515484,0.420444,0.488364,0.219766,0.335205,0.074536,0.500632,0.335905
2022-06-06,0.340894,0.410881,0.18925,0.48557,0.485429,0.522978,0.535036,0.519286,0.504208,0.555267,0.546713,0.460909,0.606165,0.481857,0.613075,0.306235,0.426049,0.121359,0.554311,0.437263
2022-06-16,0.332215,0.404776,0.200298,0.469291,0.462029,0.501208,0.515248,0.499641,0.476424,0.546185,0.53971,0.518927,0.62607,0.487566,0.620358,0.310693,0.413196,0.09317,0.52428,0.466243
2022-06-18,0.332693,0.38692,0.23444,0.570678,0.570435,0.593128,0.614091,0.605308,0.596507,0.622522,0.597883,0.605729,0.640256,0.514091,0.682154,0.40384,0.508536,0.198687,0.62721,0.472738
